In [1]:
import pandas as pd
from importlib.machinery import SourceFileLoader
from datetime import datetime as dt
import hashlib
import sys
import os
from pathlib import Path
file = Path(os.getcwd()).resolve()
parent = file.parent
sys.path.append(str(parent))

from workers import sql_stooq as sql


In [2]:
dat=pd.read_csv("tab3.csv")
tab = "INDEXES"
db_file = '../stooq.sqlite'
dat['tab']=tab

In [3]:
dat["HASH"] = [
        hashlib.md5("".join(r).encode("utf-8")).hexdigest()
        for r in dat.loc[:, ["symbol", "name", "tab"]].to_records(index=False)
    ]
dat.drop(columns=['tab'], inplace=True)

In [4]:
sql_columns = sql.tab_columns(tab,db_file)
match_columns = [col in sql_columns for col in dat.columns]
if not all(match_columns):
    print(f"Not known column {list(dat.columns[[not c for c in match_columns]])} in table. Can not write to sql.")

In [10]:

records = list(dat.to_records(index=False, column_dtypes=pd.StringDtype))

In [6]:
cmd=[f"""INSERT OR REPLACE INTO {tab} {tuple(dat.columns)}
VALUES {values_list}
""" for values_list in records]